# TP — Interprétation des Filtres CNN & Grad-CAM
**Master TRIED** · Conférence Ouverture Professionnelle — Reconnaissance faciale avec VGG16

> **Fil conducteur** : Les CNN sont souvent perçus comme des « boîtes noires ». Ce TP démonte cette idée en visualisant ce que le réseau apprend (Partie 1), comment il prend ses décisions (Partie 2), et comment exploiter ces connaissances pour construire un système pratique (Parties 3-4).

---
## 0. Environnement

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2, os

# Reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

# Téléchargement automatique de l'image (pour Colab)
if not os.path.exists("test_face.jpg"):
    print("📥 Téléchargement de l'image de test...")
    os.system("wget https://raw.githubusercontent.com/Pchambet/cnn-explainability-workshop/main/test_face.jpg")

os.makedirs("output_figures", exist_ok=True)
print(f"TensorFlow {tf.__version__} · NumPy {np.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU') or 'CPU uniquement'}")

---
# Partie 1 — Visualisation des Filtres par Maximisation des Activations

## 1.1 Principe de l'algorithme

La **maximisation des activations** (Feature Visualization) répond à une question simple mais profonde :

> *Quel stimulus visuel maximise la réponse d'un neurone donné ?*

Formellement, on cherche l'image $x^*$ qui maximise l'activation $A_{l,f}$ d'un filtre $f$ dans la couche $l$ :

$$x^* = \arg\max_x \; \frac{1}{HW} \sum_{i,j} A_{l,f}(x)_{i,j}$$

C'est un problème d'**optimisation dans l'espace des pixels** (et non des poids). On part d'une image de bruit aléatoire et on applique du **gradient ascent** :

$$x_{t+1} = x_t + \eta \cdot \frac{\nabla_x A_{l,f}}{\|\nabla_x A_{l,f}\|_2}$$

La normalisation L2 du gradient est cruciale : elle stabilise l'optimisation en assurant un pas de taille constante, quelle que soit l'amplitude du gradient.

> **Intuition** : C'est comme demander au réseau de « rêver » — on lui fait générer l'image idéale qu'il associe à un concept appris.

> Ref: [Visualizing what convnets learn — Keras](https://keras.io/examples/vision/visualizing_what_convnets_learn/)

## 1.2 Chargement et analyse de VGG16

In [ ]:
# Deux versions du modèle pour deux usages distincts :
# 1) model : include_top=True (224×224 fixe) → classification, Grad-CAM, prédictions
# 2) model_notop : include_top=False (taille flexible) → visualisation des filtres
#    (la visualisation génère des images 128×128, incompatible avec l'entrée fixe 224×224)

model = keras.applications.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
model_notop = keras.applications.VGG16(weights='imagenet', include_top=False)

# Analyse structurelle par bloc
print("=" * 70)
print("VGG16 — ARCHITECTURE PAR BLOCS")
print("=" * 70)
total_conv, total_fc = 0, 0
for layer in model.layers:
    cfg = layer.get_config()
    if 'conv' in layer.name:
        n = cfg.get('filters', 0)
        total_conv += n
        print(f"  CONV  {layer.name:20s} | {n:>3} filtres | kernel {cfg.get('kernel_size')}")
    elif 'pool' in layer.name:
        print(f"  POOL  {layer.name:20s} | ↓ /2")
    elif 'dense' in layer.name or 'predictions' in layer.name:
        u = cfg.get('units', 0)
        total_fc += u
        print(f"  FC    {layer.name:20s} | {u:>5} unités")

print(f"\nTotal conv filtres: {total_conv}")
print(f"Total FC unités:    {total_fc}")
print(f"Total paramètres:   {model.count_params():,}")

# Ratio paramètres conv vs FC
conv_params = sum(l.count_params() for l in model.layers if 'conv' in l.name)
fc_params = sum(l.count_params() for l in model.layers if 'dense' in l.name or 'predictions' in l.name)
print(f"\n⚠️  Paramètres FC: {fc_params:,} ({fc_params/model.count_params()*100:.1f}%) vs Conv: {conv_params:,} ({conv_params/model.count_params()*100:.1f}%)")
print("→ Les couches FC représentent la majorité des paramètres mais PAS la majorité de la connaissance visuelle.")

### Observation clé : le paradoxe des paramètres VGG16

Les couches **fully-connected** contiennent ~89% des paramètres, mais ce sont les couches **convolutionnelles** (~11%) qui encodent la connaissance visuelle. Ce paradoxe explique pourquoi le **transfer learning** fonctionne : on réutilise les couches conv (extracteur de features universel) et on remplace les FC (classifieur spécifique au domaine).

**Architecture en pyramide inversée** :
- Résolution spatiale : 224→112→56→28→14→7 (↓ ×32)
- Nombre de filtres : 64→128→256→512→512 (↑ ×8)
- → Le réseau **échange de la résolution spatiale contre de la profondeur sémantique**

## 1.3 Visualisation des filtres

In [ ]:
def visualize_filter(layer_name, filter_index, size=128, steps=30, lr=10.0):
    """
    Génère l'image qui maximise l'activation d'un filtre.
    Utilise model_notop (entrée flexible) pour accepter des tailles arbitraires.
    """
    extractor = keras.Model(
        inputs=model_notop.input,
        outputs=model_notop.get_layer(layer_name).output
    )
    image = tf.Variable(tf.random.uniform((1, size, size, 3)) * 0.25 + 0.5)
    
    for _ in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(image)
            activation = extractor(image)
            # Crop les bords pour éviter les artefacts de bord
            filter_activation = activation[:, 2:-2, 2:-2, filter_index]
            loss = tf.reduce_mean(filter_activation)
        grads = tape.gradient(loss, image)
        grads = tf.math.l2_normalize(grads)
        image.assign_add(lr * grads)
    
    img = image[0].numpy()
    img = (img - img.mean()) / (img.std() + 1e-5) * 0.15 + 0.5
    return np.clip(img, 0, 1)

# Validation rapide
test = visualize_filter('block1_conv2', 0, steps=5)
print(f"✅ Fonction opérationnelle — shape: {test.shape}")

### Visualisation sur 3 niveaux hiérarchiques

On visualise 8 filtres par couche à 3 niveaux de profondeur pour observer la **hiérarchie des représentations** :

| Niveau | Couche | Profondeur | Champ réceptif théorique |
|--------|--------|:----------:|:------------------------:|
| Bas | `block1_conv2` | 2 conv | 5×5 px |
| Intermédiaire | `block3_conv3` | 8 conv | 44×44 px |
| Haut | `block5_conv3` | 13 conv | 196×196 px |

Le **champ réceptif** croissant explique pourquoi les couches profondes détectent des structures de plus en plus globales.

In [ ]:
layers = [
    ("block1_conv2", "Bas niveau"),
    ("block3_conv3", "Intermédiaire"),
    ("block5_conv3", "Haut niveau"),
]
N_FILTERS = 8

fig, axes = plt.subplots(len(layers), N_FILTERS, figsize=(20, 8))
for row, (name, label) in enumerate(layers):
    print(f"⏳ {name}...")
    for col in range(N_FILTERS):
        img = visualize_filter(name, col)
        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        if col == 0:
            axes[row, col].set_ylabel(label, fontsize=10, rotation=0, labelpad=130, fontweight='bold')

plt.suptitle("Filtres VGG16 — Maximisation des activations (3 niveaux)", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("output_figures/01_filters.png", dpi=150, bbox_inches='tight')
plt.show()

## 1.4 Analyse quantitative des filtres

In [ ]:
# Mesurer la diversité et la qualité des filtres par couche
def analyze_filters(layer_name, n_filters=16, size=64, steps=20):
    """Analyse statistique des filtres d'une couche."""
    images = []
    for i in range(n_filters):
        img = visualize_filter(layer_name, i, size=size, steps=steps)
        images.append(img)
    images = np.array(images)
    
    # 1) Entropie moyenne (complexité visuelle)
    entropies = []
    for img in images:
        gray = np.mean(img, axis=2)
        hist, _ = np.histogram(gray, bins=50)
        hist = hist / (hist.sum() + 1e-10)  # Normalize to probabilities
        hist = hist[hist > 0]
        entropies.append(-np.sum(hist * np.log2(hist)))
    
    # 2) Corrélation inter-filtres (diversité)
    flat = images.reshape(n_filters, -1)
    corr_matrix = np.corrcoef(flat)
    # Moyenne des corrélations hors-diagonale
    mask = ~np.eye(n_filters, dtype=bool)
    mean_corr = np.abs(corr_matrix[mask]).mean()
    
    # 3) Variance spatiale (structure vs bruit)
    variances = [np.var(img) for img in images]
    
    return {
        'entropy_mean': np.mean(entropies),
        'entropy_std': np.std(entropies),
        'inter_correlation': mean_corr,
        'spatial_variance': np.mean(variances),
        'n_analyzed': n_filters
    }

print("Analyse quantitative des filtres (peut prendre 2-3 min)...")
results = {}
for name, label in [("block1_conv2", "Bas"), ("block3_conv3", "Mid"), ("block5_conv3", "Haut")]:
    print(f"  ⏳ {name}...")
    results[name] = analyze_filters(name)

# Affichage
print("\n" + "=" * 70)
print(f"{'Couche':20s} | {'Entropie':>10s} | {'Corr. inter':>12s} | {'Variance':>10s}")
print("-" * 70)
for name, r in results.items():
    print(f"{name:20s} | {r['entropy_mean']:>7.2f}±{r['entropy_std']:.2f} | {r['inter_correlation']:>11.4f} | {r['spatial_variance']:>10.6f}")
print("=" * 70)

### 1.5 Discussion approfondie

#### Hiérarchie des représentations — Pourquoi c'est important

Les résultats confirment expérimentalement une propriété fondamentale des CNNs profonds :

**Couches basses (block1)** :
- Entropie **faible** → motifs simples et bien structurés (bords, gradients)
- Corrélation inter-filtres **faible** → chaque filtre capture un concept visuel distinct
- Ces filtres sont des **détecteurs de Gabor** appris — ils redécouvrent les primitives identifiées par les neurosciences (Hubel & Wiesel, 1962) dans le cortex visuel V1

**Couches intermédiaires (block3)** :
- Entropie **croissante** → motifs plus complexes (textures, motifs répétitifs)
- Les filtres commencent à **composer** les primitives des couches basses
- Analogie biologique : aires V2/V4 du cortex visuel

**Couches hautes (block5)** :
- Entropie **maximale** → structures très complexes, parfois difficiles à interpréter visuellement
- Corrélation inter-filtres **plus élevée** → les filtres se spécialisent sur des variations d'un même concept
- Certains filtres semblent « bruités » → ce n'est pas du bruit, c'est un **pattern trop abstrait** pour notre perception

#### Impact du learning rate sur l'entraînement

| LR | Effet sur les filtres | Diagnostic |
|----|----------------------|------------|
| Trop élevé | Filtres bruités, pas de structure → oscillation des poids | « Salt & pepper » pattern |
| Optimal | Filtres nets, diversifiés, hiérarchisés | Structure claire à chaque couche |
| Trop bas | Filtres redondants ou « morts » → convergence insuffisante | Beaucoup de filtres quasi-identiques |

#### Transférabilité des filtres

Les filtres bas-niveau sont **quasi-universels** (bords, textures) : on les retrouve dans des réseaux entraînés sur ImageNet, sur des visages, ou même sur des images médicales. C'est le fondement du **transfer learning** — on peut réutiliser ces couches et ne ré-entraîner que les couches hautes sur un nouveau domaine.

> 📚 Yosinski et al. (2014), *"How transferable are features in deep neural networks?"* — les 3 premières couches sont quasi-identiques entre réseaux entraînés sur des tâches différentes.

---
# Partie 2 — Grad-CAM, Occlusion & Anonymisation

## 2.1 Grad-CAM : théorie

**Grad-CAM** produit une carte de chaleur des régions qui influencent la décision du réseau. Contrairement à la visualisation des filtres (Partie 1) qui montre ce que le réseau *peut* détecter, le Grad-CAM montre ce qu'il *utilise effectivement* pour une image donnée.

**Formulation mathématique** :

1. On calcule les poids d'importance $\alpha_k^c$ pour chaque feature map $A^k$ :
$$\alpha_k^c = \underbrace{\frac{1}{Z} \sum_{i} \sum_{j}}_{\text{Global Average Pooling}} \frac{\partial y^c}{\partial A^k_{ij}}$$

2. La carte d'activation est :
$$L^c_{\text{Grad-CAM}} = \text{ReLU}\left(\sum_k \alpha_k^c \cdot A^k\right)$$

Le **ReLU** ne conserve que les activations *positives* : on veut les régions qui **contribuent** à la classe, pas celles qui la **contredisent**.

> Ref: Selvaraju et al. (2017), [*"Grad-CAM: Visual Explanations from Deep Networks"*](https://arxiv.org/abs/1610.02391)

## 2.2 Implémentation

In [ ]:
IMG_PATH = "test_face.jpg"

def load_image(path, size=(224, 224)):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    x = keras.applications.vgg16.preprocess_input(
        np.expand_dims(img.astype('float32').copy(), 0)
    )
    return img, x

def make_gradcam_heatmap(img_array, model, layer_name, pred_index=None):
    grad_model = keras.Model(
        inputs=model.input,
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, conv_out)
    pooled = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.squeeze(conv_out[0] @ pooled[..., tf.newaxis])
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def overlay_heatmap(img, heatmap, alpha=0.4):
    h = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    h_color = cv2.applyColorMap(np.uint8(255 * h), cv2.COLORMAP_JET)
    h_color = cv2.cvtColor(h_color, cv2.COLOR_BGR2RGB)
    return (h_color * alpha + img * (1 - alpha)).astype(np.uint8)

img_rgb, img_input = load_image(IMG_PATH)
print("✅ Fonctions définies, image chargée")

In [ ]:
# Prédictions ImageNet
preds = model.predict(img_input, verbose=0)
top5 = keras.applications.vgg16.decode_predictions(preds, top=5)[0]
print("Top 5 prédictions ImageNet :")
for i, (_, label, score) in enumerate(top5):
    bar = "█" * int(score * 50)
    print(f"  {i+1}. {label:20s} {score:.4f} {bar}")

print(f"\n⚠️  Le modèle est entraîné sur ImageNet (objets), pas VGGFace (visages).")
print("   → Il classifie le vêtement (jersey), pas l'identité de la personne.")
print("   → C'est un point clé pour l'interprétation du Grad-CAM ci-dessous.")

## 2.3 Grad-CAM multi-couches

In [ ]:
cam_layers = ['block3_conv3', 'block4_conv3', 'block5_conv3']

fig, axes = plt.subplots(2, len(cam_layers) + 1, figsize=(20, 10))

# Row 1: images + overlays
axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title("Original", fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

heatmaps = {}
for i, layer in enumerate(cam_layers):
    hm = make_gradcam_heatmap(img_input, model, layer)
    heatmaps[layer] = hm
    axes[0, i+1].imshow(overlay_heatmap(img_rgb, hm))
    axes[0, i+1].set_title(f"Grad-CAM: {layer}", fontsize=12, fontweight='bold')
    axes[0, i+1].axis('off')

# Row 2: raw heatmaps (pour analyse quantitative)
axes[1, 0].text(0.5, 0.5, "Heatmaps\nbrutes\n(sans overlay)", 
                ha='center', va='center', fontsize=12, transform=axes[1,0].transAxes)
axes[1, 0].axis('off')

for i, layer in enumerate(cam_layers):
    im = axes[1, i+1].imshow(heatmaps[layer], cmap='jet', vmin=0, vmax=1)
    axes[1, i+1].set_title(f"Raw: {layer}", fontsize=11)
    axes[1, i+1].axis('off')
    plt.colorbar(im, ax=axes[1, i+1], fraction=0.046)

plt.suptitle("Grad-CAM — Évolution de l'attention à travers les couches", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("output_figures/02_gradcam.png", dpi=150, bbox_inches='tight')
plt.show()

## 2.4 Analyse quantitative du Grad-CAM

In [ ]:
# Définir des régions d'intérêt (ROI) sur le visage
rois = {
    "Front/Cheveux": (0, 0.25, 0, 1),      # (y1%, y2%, x1%, x2%)
    "Yeux":          (0.25, 0.42, 0.1, 0.9),
    "Nez":           (0.42, 0.58, 0.25, 0.75),
    "Bouche/Menton": (0.58, 0.78, 0.15, 0.85),
    "Cou/Vêtement":  (0.78, 1.0, 0, 1),
}

print("=" * 75)
print(f"{'Région':20s}", end="")
for layer in cam_layers:
    print(f" | {layer:>15s}", end="")
print("\n" + "-" * 75)

for roi_name, (y1, y2, x1, x2) in rois.items():
    print(f"{roi_name:20s}", end="")
    for layer in cam_layers:
        hm = heatmaps[layer]
        h, w = hm.shape
        roi_val = hm[int(h*y1):int(h*y2), int(w*x1):int(w*x2)].mean()
        bar = "█" * int(roi_val * 20)
        print(f" | {roi_val:>6.3f} {bar:10s}", end="")
    print()

print("=" * 75)
print("\n→ Valeurs plus élevées = la région contribue plus à la classification")

### Analyse critique des résultats Grad-CAM

#### Observation principale : le biais ImageNet

Le modèle prédit **"jersey"** (vêtement), pas une identité faciale. Le Grad-CAM reflète ce biais :
- **block5_conv3** montre une forte activation sur le **cou et le t-shirt** — cohérent avec la classe prédite
- **block3_conv3** diffuse l'attention sur le **visage et le col** — les textures du tissu ET de la peau sont captées
- Le réseau ne « regarde » pas le visage pour identifier une personne, mais pour classifier un objet

#### Lien avec la Partie 1

La Partie 1 nous a montré que les filtres de `block5` détectent des **structures abstraites**. Le Grad-CAM révèle maintenant *lesquels* de ces filtres s'activent sur cette image. C'est complémentaire :
- **Partie 1** = *que peut détecter le réseau ?* (capacité)
- **Partie 2** = *que détecte-t-il effectivement ?* (utilisation)

#### Implication pour un modèle VGGFace

Avec un modèle entraîné sur des visages (VGGFace), le Grad-CAM montrerait une attention concentrée sur les **traits du visage discriminants** (yeux, nez, bouche), pas sur les vêtements. Cette différence illustre que **l'interprétabilité dépend fortement du domaine d'entraînement**.

## 2.5 Occluding Parts — Analyse par occlusion systématique

L'analyse par occlusion est complémentaire au Grad-CAM : au lieu de calculer des gradients, on **masque physiquement** une zone de l'image et on mesure la chute de confiance. C'est plus coûteux mais plus intuitif et ne dépend pas d'hypothèses de linéarité.

**Protocole** : un patch gris (128, 128, 128) de 25×25 px glisse sur l'image avec un stride de 14 px. Pour chaque position, on mesure la baisse de score de la classe prédite.

In [ ]:
def occlusion_sensitivity(img_path, model, patch=25, stride=14):
    img, x = load_image(img_path)
    base_pred = model.predict(x, verbose=0)
    top_class = np.argmax(base_pred[0])
    base_score = base_pred[0][top_class]
    
    h, w = 224, 224
    smap, count = np.zeros((h, w)), np.zeros((h, w))
    
    for y in range(0, h - patch, stride):
        for x_pos in range(0, w - patch, stride):
            occ = img.copy()
            occ[y:y+patch, x_pos:x_pos+patch] = 128
            occ_input = keras.applications.vgg16.preprocess_input(
                np.expand_dims(occ.astype('float32'), 0)
            )
            score = model.predict(occ_input, verbose=0)[0][top_class]
            smap[y:y+patch, x_pos:x_pos+patch] += base_score - score
            count[y:y+patch, x_pos:x_pos+patch] += 1
    
    count[count == 0] = 1
    return smap / count, base_score

print("⏳ Analyse par occlusion (~1-2 min sur CPU)...")
smap, base_score = occlusion_sensitivity(IMG_PATH, model, patch=25, stride=14)
print(f"✅ Score de base: {base_score:.4f}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(img_rgb)
axes[0].set_title("Image originale", fontweight='bold', fontsize=12)
axes[0].axis('off')

im = axes[1].imshow(smap, cmap='hot', interpolation='bilinear')
axes[1].set_title("Carte de sensibilité (Δ confiance)", fontweight='bold', fontsize=12)
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], fraction=0.046, label="Chute de score")

axes[2].imshow(img_rgb, alpha=0.6)
axes[2].imshow(smap, cmap='hot', alpha=0.5, interpolation='bilinear')
axes[2].set_title("Superposition", fontweight='bold', fontsize=12)
axes[2].axis('off')

plt.suptitle("Occluding Parts — Quelles zones sont critiques pour la classification ?", fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig("output_figures/03_occlusion.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Analyse par zone de la carte de sensibilité
print("Sensibilité par zone anatomique :")
print("=" * 55)
for roi_name, (y1, y2, x1, x2) in rois.items():
    h, w = smap.shape
    roi_val = smap[int(h*y1):int(h*y2), int(w*x1):int(w*x2)]
    mean_s = roi_val.mean()
    max_s = roi_val.max()
    bar = "█" * int(abs(mean_s) / (abs(smap).max() + 1e-8) * 20)
    print(f"  {roi_name:20s} | mean={mean_s:>+.4f} | max={max_s:>+.4f} | {bar}")
print("=" * 55)
print(f"\nSensibilité globale max: {smap.max():.4f}")
print(f"Zone la plus sensible: {max(rois.items(), key=lambda x: smap[int(224*x[1][0]):int(224*x[1][1]), int(224*x[1][2]):int(224*x[1][3])].mean())[0]}")

### Comparaison Grad-CAM vs Occlusion

| Méthode | Avantages | Limites |
|---------|-----------|---------|
| **Grad-CAM** | Rapide (1 forward + 1 backward), résolution de la feature map | Dépend de la linéarité locale, 1 seule couche |
| **Occlusion** | Model-agnostic, intuitif, mesure directe de l'impact | Lent (N² forward passes), résolution du patch |

Les deux méthodes convergent sur les mêmes conclusions — ce qui renforce la fiabilité de l'analyse.

## 2.6 Anonymisation CNIL — Expérimentation multi-masques

La CNIL recommande le masquage des yeux pour l'anonymisation. Testons systématiquement **3 niveaux** de masquage pour quantifier leur efficacité :

In [ ]:
def apply_mask(img, mask_type):
    masked = img.copy()
    h = 224
    if mask_type == "yeux":
        masked[int(h*0.28):int(h*0.42), :] = 0
    elif mask_type == "yeux+nez":
        masked[int(h*0.28):int(h*0.58), :] = 0
    elif mask_type == "visage_complet":
        masked[int(h*0.15):int(h*0.80), int(h*0.10):int(h*0.90)] = 0
    elif mask_type == "flou_gaussien":
        face_region = masked[int(h*0.10):int(h*0.85), int(h*0.05):int(h*0.95)]
        blurred = cv2.GaussianBlur(face_region, (51, 51), 30)
        masked[int(h*0.10):int(h*0.85), int(h*0.05):int(h*0.95)] = blurred
    return masked

mask_types = ["yeux", "yeux+nez", "visage_complet", "flou_gaussien"]
fig, axes = plt.subplots(1, len(mask_types) + 1, figsize=(22, 5))

# Original
pred_orig = model.predict(img_input, verbose=0)
orig_top = keras.applications.vgg16.decode_predictions(pred_orig, top=1)[0][0]
axes[0].imshow(img_rgb)
axes[0].set_title(f"Original\n{orig_top[1]}: {orig_top[2]:.3f}", fontweight='bold')
axes[0].axis('off')

print("Résultats multi-masques :")
print("=" * 60)
print(f"  {'Masque':20s} | {'Top classe':15s} | {'Score':>8s} | {'Chute':>8s}")
print("-" * 60)

for i, mt in enumerate(mask_types):
    masked = apply_mask(img_rgb, mt)
    m_input = keras.applications.vgg16.preprocess_input(
        np.expand_dims(masked.astype('float32'), 0)
    )
    m_pred = model.predict(m_input, verbose=0)
    m_top = keras.applications.vgg16.decode_predictions(m_pred, top=1)[0][0]
    drop = orig_top[2] - m_top[2]
    
    axes[i+1].imshow(masked)
    axes[i+1].set_title(f"{mt}\n{m_top[1]}: {m_top[2]:.3f}", fontweight='bold')
    axes[i+1].axis('off')
    
    print(f"  {mt:20s} | {m_top[1]:15s} | {m_top[2]:>8.4f} | {drop:>+8.4f}")

print("=" * 60)
plt.suptitle("Efficacité comparée des méthodes d'anonymisation", fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig("output_figures/04_cnil_mask.png", dpi=150, bbox_inches='tight')
plt.show()

### 2.7 Discussion — Anonymisation & Biais Ethnique

#### Le masque sur les yeux est-il suffisant ?

Nos résultats expérimentaux montrent clairement que **non** :
- Le masque « yeux seuls » provoque une **chute de confiance faible** — le réseau continue à classifier correctement
- Le Grad-CAM confirme que le réseau utilise **nez, bouche, mâchoire, texture de la peau, et même les vêtements**
- Seul le masquage du **visage complet** ou le **flou gaussien** dégradent significativement la classification

#### Le biais ethnique : un problème systémique

L'étude de **Shrutin et al. (2019)**, *"Deep Learning for Face Recognition: Pride or Prejudiced?"*, révèle un phénomène alarmant :

1. **Répartition des features discriminantes** : pour les sujets caucasiens, le réseau concentre ~60% de son attention sur les yeux. Pour d'autres ethnicités, la distribution est plus diffuse (nez, bouche, texture)
2. **Conséquence directe** : le masquage des yeux "anonymise" efficacement les caucasiens mais **pas les autres populations**
3. **Cause racine** : les datasets d'entraînement (LFW, VGGFace) sont déséquilibrés (>70% caucasiens) → le réseau apprend des raccourcis biaisés

#### Proposition d'anonymisation robuste

| Méthode | Résistance CNN | Commodité | Recommandation |
|---------|:-:|:-:|---------------|
| Masque yeux | ❌ Faible | ✅ Simple | Insuffisante |
| Masque yeux+nez+bouche | ⚠️ Moyenne | ⚠️ Moyen | Minimum viable |
| Flou gaussien visage | ✅ Forte | ✅ Simple | **Recommandée** |
| Remplacement par avatar | ✅ Forte | ❌ Complexe | Idéale si faisable |

> **Recommandation** : le flou gaussien de l'ensemble du visage est le meilleur compromis efficacité/simplicité. Il détruit les features **à tous les niveaux** du CNN (bords, textures, structures).

---
# Partie 3 — Système de Reconnaissance Faciale One-Shot

## 3.1 Architecture & Workflow

**Contrainte** : reconnaissance avec **1 seule photo** par personne (one-shot learning)

```
┌──────────────────────────────────────────────────────────────┐
│                     ENREGISTREMENT                           │
│                                                              │
│  Photo ──→ Détection ──→ Resize ──→ VGG16 ──→ Embedding ──→ DB  │
│            visage        224×224    (frozen)   Rⁿ            │
└──────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────┐
│                     RECONNAISSANCE                           │
│                                                              │
│  Photo ──→ même pipeline ──→ Embedding ──→ Cosine Sim. ──→ ID   │
│                               requête      vs DB        ou ❌    │
└──────────────────────────────────────────────────────────────┘
```

**Choix techniques justifiés** :
- **VGG16 block5_pool** comme extracteur : produit un embedding de 25 088 dimensions (7×7×512)
- **Cosine similarity** plutôt qu'euclidienne : invariant à la norme, plus robuste aux variations d'éclairage
- **Seuil de rejet** : en dessous d'un seuil, on refuse l'identification (sécurité)

## 3.2 Implémentation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

feature_extractor = keras.Model(
    inputs=model.input,
    outputs=model.get_layer('block5_pool').output
)
print(f"Extracteur: {model.input_shape} → {feature_extractor.output_shape}")
print(f"Dimension embedding: {np.prod(feature_extractor.output_shape[1:]):,}")


class FaceRecognizer:
    def __init__(self, extractor, threshold=0.5):
        self.extractor = extractor
        self.threshold = threshold
        self.db = {}
    
    def _embed(self, img_path):
        _, x = load_image(img_path)
        feat = self.extractor.predict(x, verbose=0).flatten()
        return feat / (np.linalg.norm(feat) + 1e-8)
    
    def register(self, name, img_path):
        self.db[name] = self._embed(img_path)
        print(f"  ✅ '{name}' enregistré (dim={len(self.db[name])})")
    
    def identify(self, img_path):
        if not self.db:
            return "DB vide", 0.0
        query = self._embed(img_path)
        best = max(self.db.items(), key=lambda item: np.dot(query, item[1]))
        sim = np.dot(query, best[1])
        status = "✅ Identifié" if sim >= self.threshold else "❌ Rejeté"
        return best[0], sim, status

# Démonstration
recognizer = FaceRecognizer(feature_extractor)
recognizer.register("Personne Test", IMG_PATH)

name, score, status = recognizer.identify(IMG_PATH)
print(f"\n🔍 Test self-identification:")
print(f"   Résultat: {name} | Cosine sim: {score:.6f} | {status}")
print(f"\n   → sim=1.0 attendue (même image) : {'✅ Correct' if score > 0.999 else '⚠️ Inattendu'}")

## 3.3 Analyse — Pourquoi KNN et pas un réseau de neurones ?

#### Le dilemme du one-shot

Un classificateur NN classique nécessite **des centaines d'exemples** par classe pour apprendre les frontières de décision. Avec une seule image par personne, il **overfitterait** immédiatement.

Le **KNN (K=1)** n'a pas ce problème car il ne « s'entraîne » pas — il compare directement les embeddings dans l'espace des features.

#### Pourquoi ça fonctionne

Le succès repose entièrement sur la **qualité de l'espace d'embedding** :
1. VGG16 (pré-entraîné) projette les images dans un espace où les **distances sémantiques** sont préservées
2. Deux photos de la même personne → embeddings proches (haute cosine sim.)
3. Photos de personnes différentes → embeddings éloignés

C'est du **metric learning implicite** : le réseau n'a pas été entraîné explicitement pour la similarité, mais les features de haut niveau capturent naturellement l'identité.

#### Limites et améliorations

| Limite | Solution |
|--------|----------|
| VGG16 ImageNet ≠ visages | Utiliser VGGFace ou ArcFace |
| Embedding 25K dims → lent | PCA ou auto-encoder pour compression |
| Pas robuste aux poses extrêmes | Data augmentation à l'enregistrement |
| Seuil fixe | Seuil adaptatif par personne |

---
# Partie 4 — Production & Déploiement

## 4.1 Export du modèle

In [ ]:
save_path = "saved_model/face_features"
feature_extractor.export(save_path)

import subprocess
result = subprocess.run(['du', '-sh', save_path], capture_output=True, text=True)
print(f"✅ Modèle exporté: {result.stdout.strip()}")
print(f"   Format: TensorFlow SavedModel (.pb + variables)")

## 4.2 Contraintes de production

| Contrainte | VGG16 brut | Après optimisation |
|------------|:----------:|:------------------:|
| **Taille modèle** | ~528 MB | ~60 MB (quantization INT8) |
| **Latence CPU** | ~200ms | ~50ms (TF Lite + XNNPACK) |
| **Latence GPU** | ~15ms | ~5ms (TensorRT) |
| **RAM inference** | ~1.5 GB | ~200 MB |

#### Options de déploiement

1. **Serveur (TF Serving)** : API REST/gRPC, batch processing, scalable
2. **Mobile (TF Lite)** : quantification INT8, délégué GPU, ~60 MB
3. **Navigateur (TF.js)** : WebGL backend, pas de serveur, ~15 MB (après pruning)
4. **Embarqué (C++ / OpenCV DNN)** : performance native, pas de dépendances Python

#### Sécurité & RGPD

- Les **embeddings faciaux sont des données biométriques** → Article 9 RGPD
- Chiffrement obligatoire au repos et en transit
- Droit à l'effacement : supprimer l'embedding = "oublier" une personne
- **Pas de stockage des images originales** en production, uniquement les embeddings

---
# Conclusion & Perspectives

## Synthèse des résultats

| Question | Méthode utilisée | Réponse |
|----------|-----------------|---------|
| Que détectent les filtres CNN ? | Maximisation des activations | Hiérarchie bords → textures → structures |
| Quelles zones influencent la décision ? | Grad-CAM + Occlusion | Dépend du domaine d'entraînement (ImageNet ≠ VGGFace) |
| Le masque yeux suffit-il ? | Multi-masques + analyse quantitative | **Non** — seul le flou complet est robuste |
| KNN vs NN en one-shot ? | Analyse théorique + implémentation | KNN + transfer learning est optimal |

## Ce que ce TP révèle sur les CNNs

1. **Les CNNs ne sont pas des boîtes noires** — on dispose d'outils (filter viz, Grad-CAM, occlusion) pour comprendre leurs décisions
2. **L'interprétabilité est un outil pratique** — elle permet de questionner des pratiques réelles (CNIL) avec des preuves visuelles
3. **Le domaine d'entraînement est déterminant** — un même réseau (VGG16) « regarde » des zones différentes selon qu'il a appris des objets ou des visages
4. **Le biais est structurel** — il vient des données, pas de l'algorithme, et se propage silencieusement dans les recommandations réglementaires

## Pour aller plus loin

- Comparer les Grad-CAM de VGG16-ImageNet vs VGGFace sur la même image
- Tester avec [ArcFace](https://arxiv.org/abs/1801.07698) — l'état de l'art en reconnaissance faciale
- Ajouter l'analyse SHAP pour une attribution pixel-level
- Implémenter un test A/B multi-ethnicité pour quantifier le biais